In [1]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import os
import json
from datetime import datetime
from os import walk

In [2]:
es = Elasticsearch(
    cloud_id='MarathonDuWeb:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGZhMzJkNzYyNzFjYzQ4OTRhNjk2ZWQ4NTVhYjkwMDgyJGZjOTk1YzA0ZmYyYjQ5ZmM4OTliYjAwODNkYzA5N2U0',
    basic_auth=('elastic', 'QjTCAIOmAkkqqTIGjeEeD63q')
)

In [3]:
listeFichiers = []
monRepertoire = '../Data/MoyenJH'
for (repertoire, sousRepertoires, fichiers) in walk(monRepertoire):
     listeFichiers.extend(fichiers)
listeFichiers
for fichier in listeFichiers:
    path_fichier = monRepertoire+'/'+fichier
    data = pd.read_csv(path_fichier)
    nom_fichier = os.path.basename(path_fichier) 
    if len(nom_fichier)==19:
        nom_parking = nom_fichier[len(nom_fichier)-8:len(nom_fichier)-4]
    else:
        nom_parking = nom_fichier[len(nom_fichier)-9:len(nom_fichier)-4]
    for row in data.index:
        data['Heure'][row] = data['Heure'][row][0:2]
        date_time_obj = datetime.strptime(data['Heure'][row] ,'%H')
        data['Heure'][row] = date_time_obj.time()
    aggregation_functions = {'Heure': 'first', 'Dimanche': 'mean', 'Jeudi': 'mean', 'Lundi':'mean','Mardi':'mean','Mercredi':'mean','Samedi':'mean','Vendredi':'mean'}
    data = data.groupby(data['Heure']).aggregate(aggregation_functions)
    data['Heure'] = data['Heure'].astype(str)
    dic_to_es = {}
    dic_to_es  = {'Parking' : nom_parking, 'Jours' : []}
    for column in data[['Dimanche','Jeudi','Lundi','Mardi','Mercredi','Samedi','Vendredi']]:
        temp_jour_dic = {'Jour' : column, 'Données': []}
        for row in data.index:
            temp_data_dic = {'Heure': data['Heure'][row], 'Places_libres':data[column][row]}
            temp_jour_dic['Données'].append(temp_data_dic)
        dic_to_es['Jours'].append(temp_jour_dic)
    with open('../MontpelYeah/static/data/'+nom_parking+'.json','w') as fp:
        json.dump(dic_to_es,fp)
    #es.index(
        #index='moyenne_jour_index',
        #document=dic_to_es)
    
    
   

<ipython-input-3-5c5131b72d04>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Heure'][row] = data['Heure'][row][0:2]
<ipython-input-3-5c5131b72d04>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Heure'][row] = date_time_obj.time()


data = pd.read_csv('../Data/MoyenJH/MoyenFreeJHANTI.csv')
data

for row in data.index:
    #print(data['Heure'][row][0:2])
    data['Heure'][row] = data['Heure'][row][0:2]
    date_time_obj = datetime.strptime(data['Heure'][row] ,'%H')
    data['Heure'][row] = date_time_obj.time()
data

print(type(data['Heure'][0]))
data['Heure'][0]=data['Heure'].dt.round('60min')
data

#print(type(data['Heure'][0]))
#data = data.set_index('Heure')
data = data.set_index(pd.DatetimeIndex(data['Heure']))
print(data)
data.round('H')
#print(data)
#print(data.columns.values)
#data.resample('H',on='Heure').Dimanche.sum()


aggregation_functions = {'Heure': 'first', 'Dimanche': 'mean', 'Jeudi': 'mean', 'Lundi':'mean','Mardi':'mean','Mercredi':'mean','Samedi':'mean','Vendredi':'mean'}
data = data.groupby(data['Heure']).aggregate(aggregation_functions)
data

data['Heure'] = data['Heure'].astype(str)
type(data['Heure'][0])

nom_fichier = os.path.basename("../Data/MoyenJH/MoyenFreeJHANTI.csv")
nom_parking = nom_fichier[len(nom_fichier)-9:len(nom_fichier)-4]
nom_parking

for row in data.index:
    print(data['Heure'][row])

dic_to_es = {}
dic_to_es  = {'Parking' : nom_parking, 'Jours' : []}
for column in data[['Dimanche','Jeudi','Lundi','Mardi','Mercredi','Samedi','Vendredi']]:
    temp_jour_dic = {'Jour' : column, 'Données': []}
    for row in data.index:
        #print(data['Heure'][row])
        #print(type(data['Heure'][row]))
        temp_data_dic = {'Heure': data['Heure'][row], 'Places_libres':data[column][row]}
        temp_jour_dic['Données'].append(temp_data_dic)
    dic_to_es['Jours'].append(temp_jour_dic)
#print(dic_to_es)

dic_to_es

json_object = json.dumps(dic_to_es)
print(json_object)